In [29]:
import uproot
import awkward as ak
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplhep as hep
import os, glob
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import mplhep as hep
plt.style.use([hep.style.ROOT, hep.style.firamath])
from matplotlib.colors import LinearSegmentedColormap
# Define the CMS color scheme
cms_colors = [
    (0.00, '#FFFFFF'),  # White
    (0.33, '#005EB8'),  # Blue
    (0.66, '#FFDD00'),  # Yellow
    (1.00, '#FF0000')   # red
]

# Create the CMS colormap
cms_cmap = LinearSegmentedColormap.from_list('CMS', cms_colors)


from numpy.lib.stride_tricks import as_strided


def upsample_array(x, b0, b1):

    r, c = x.shape                                    # number of rows/columns
    rs, cs = x.strides                                # row/column strides
    x = as_strided(x, (r, b0, c, b1), (rs, 0, cs, 0)) # view as a larger 4D array

    return x.reshape(r*b0, c*b1)/(b0*b1)              # create new 2D array with same total occupancy

def resample_EE(imgECAL, factor=2):

    # EE-
    imgEEm = imgECAL[:140-85] # EE- in the first 55 rows
    imgEEm = np.pad(imgEEm, ((1,0),(0,0)), 'constant', constant_values=0) # for even downsampling, zero pad 55 -> 56
    imgEEm_dn = block_reduce(imgEEm, block_size=(factor, factor), func=np.sum) # downsample by summing over [factor, factor] window
    imgEEm_dn_up = upsample_array(imgEEm_dn, factor, factor)/(factor*factor) # upsample will use same values so need to correct scale by factor**2
    imgECAL[:140-85] = imgEEm_dn_up[1:] ## replace the old EE- rows

    # EE+
    imgEEp = imgECAL[140+85:] # EE+ in the last 55 rows
    imgEEp = np.pad(imgEEp, ((0,1),(0,0)), 'constant', constant_values=0) # for even downsampling, zero pad 55 -> 56
    imgEEp_dn = block_reduce(imgEEp, block_size=(factor, factor), func=np.sum) # downsample by summing over [factor, factor] window
    imgEEp_dn_up = upsample_array(imgEEp_dn, factor, factor)/(factor*factor) # upsample will use same values so need to correct scale by factor*factor
    imgECAL[140+85:] = imgEEp_dn_up[:-1] # replace the old EE+ rows

    return imgECAL

def crop_jet(imgECAL, iphi, ieta, jet_shape=125):

    # NOTE: jet_shape here should correspond to the one used in RHAnalyzer
    off = jet_shape//2
    iphi = int(iphi*5 + 2) # 5 EB xtals per HB tower
    ieta = int(ieta*5 + 2) # 5 EB xtals per HB tower

    # Wrap-around on left side
    if iphi < off:
        diff = off-iphi
        img_crop = np.concatenate((imgECAL[:,ieta-off:ieta+off+1,-diff:],
                                   imgECAL[:,ieta-off:ieta+off+1,:iphi+off+1]), axis=-1)
    # Wrap-around on right side
    elif 360-iphi < off:
        diff = off - (360-iphi)
        img_crop = np.concatenate((imgECAL[:,ieta-off:ieta+off+1,iphi-off:],
                                   imgECAL[:,ieta-off:ieta+off+1,:diff+1]), axis=-1)
    # Nominal case
    else:
        img_crop = imgECAL[:,ieta-off:ieta+off+1,iphi-off:iphi+off+1]

    return img_crop

In [30]:
file = uproot.open("../analysis/mass_regreesion_ntuples/CMSSW_10_6_20/src/MLAnalyzer/HToAATo4Tau_M10_correct_pixel_numEvent10.root")
RHTree = file["fevt/RHTree"]

In [34]:
layers =["ECAL_energy","HBHE_energy","TracksAtECAL_pt","TracksAtECAL_dZSig","TracksAtECAL_d0Sig","PixAtEcal_1","PixAtEcal_2",
        "PixAtEcal_3","PixAtEcal_1","TibAtEcal_1","TibAtEcal_2","TibAtEcal_3","TibAtEcal_4","TobAtEcal_1","TobAtEcal_2",
        "TobAtEcal_3","TobAtEcal_4","TobAtEcal_5","TobAtEcal_6","TecAtEcal_1","TecAtEcal_2","TecAtEcal_3","TidAtEcal_1"]
print("Number of layers", len(layers))


Number of layers 23


In [ ]:
ECAL_energy_ = np.array(rhTree.ECAL_energy)
ECAL_energy__ = ECAL_energy_.reshape(280,360)
ECAL_energy = resample_EE(ECAL_energy__)
HBHE_energy_ = np.array(rhTree.HBHE_energy)
HBHE_energy__ = HBHE_energy_.reshape(56,72)
HBHE_energy = upsample_array(HBHE_energy__, 5, 5) # (280, 360)
TracksAtECAL_pt_ = np.array(rhTree.ECAL_tracksPt_atECALfixIP)
TracksAtECAL_pt    = TracksAtECAL_pt_.reshape(280,360)
TracksAtECAL_dZSig_ = np.array(rhTree.ECAL_tracksDzSig_atECALfixIP)
TracksAtECAL_dZSig = TracksAtECAL_dZSig_.reshape(280,360)
TracksAtECAL_d0Sig_ = np.array(rhTree.ECAL_tracksD0Sig_atECALfixIP)
TracksAtECAL_d0Sig = TracksAtECAL_d0Sig_.reshape(280,360)
PixAtEcal_1        = np.array(rhTree.BPIX_layer1_ECAL_atPV).reshape(280,360)
PixAtEcal_2        = np.array(rhTree.BPIX_layer2_ECAL_atPV).reshape(280,360)
PixAtEcal_3        = np.array(rhTree.BPIX_layer3_ECAL_atPV).reshape(280,360)
PixAtEcal_4        = np.array(rhTree.BPIX_layer4_ECAL_atPV).reshape(280,360)
TibAtEcal_1        = np.array(rhTree.TIB_layer1_ECAL_atPV).reshape(280,360)
TibAtEcal_2        = np.array(rhTree.TIB_layer2_ECAL_atPV).reshape(280,360)
TibAtEcal_3        = np.array(rhTree.TIB_layer3_ECAL_atPV).reshape(280,360)
TibAtEcal_4        = np.array(rhTree.TIB_layer4_ECAL_atPV).reshape(280,360)
TobAtEcal_1        = np.array(rhTree.TOB_layer1_ECAL_atPV).reshape(280,360)
TobAtEcal_2        = np.array(rhTree.TOB_layer2_ECAL_atPV).reshape(280,360)
TobAtEcal_3        = np.array(rhTree.TOB_layer3_ECAL_atPV).reshape(280,360)
TobAtEcal_4        = np.array(rhTree.TOB_layer4_ECAL_atPV).reshape(280,360)
TobAtEcal_5        = np.array(rhTree.TOB_layer5_ECAL_atPV).reshape(280,360)
TobAtEcal_6        = np.array(rhTree.TOB_layer6_ECAL_atPV).reshape(280,360)
TecAtEcal_1        = np.array(rhTree.TEC_layer1_ECAL_atPV).reshape(280,360)
TecAtEcal_2        = np.array(rhTree.TEC_layer2_ECAL_atPV).reshape(280,360)
TecAtEcal_3        = np.array(rhTree.TEC_layer3_ECAL_atPV).reshape(280,360)
TidAtEcal_1        = np.array(rhTree.TID_layer1_ECAL_atPV).reshape(280,360)